### Problem 3.

Jades Cargo has to deliver goods using 4 fours ships from Manila to 4 different ports in Mindanao. Due to the different types of cargo that each ship carries, the total cost of loading,
transporting, unloading and further land transport of the goods would vary according to the port
where each ship would dock. The company is allowed to dock exactly one ship at each port. The cost for the trip of each ship to the different ports are as follows:

    Ship 1 to port of Zamboanga is 500,000; 
    Ship 1 to port of Iligan is 400,000; 
    Ship 1 to port of Cagayan de Oro is 600,000; 
    Ship 1 to port of Davao is 700,000; 

    Ship 2 to port of Zamboanga is 600,000; 
    Ship 2 to port of Iligan is 600,000; 
    Ship 2 to port of Cagayan de Oro is 700,000; 
    Ship 2 to port of Davao is 500,000;

    Ship 3 to port of Zamboanga is 700,000;
    Ship 3 to port of Iligan is 500,000;
    Ship 3 to port of Cagayan de Oro is 700,000;
    Ship 3 to port of Davao is 600,000;

    Ship 4 to port of Zamboanga is 500,000;
    Ship 4 to port of Iligan is 400,000;
    Ship 4 to port of Cagayan de Oro is 600,000;
    Ship 4 to port of Davao is 600,000

The management of Jades Cargo needs to know which ship should go to which port to minimize the transportation of the goods. 

#### a) Construct the mathematical model of the problem

##### Variables:

    Vij = Binary matrix denoting 1 if ship i is assign to port j, 0 if otherwise
            where i=ship number, i.e., 0=Ship1; 1=Ship2; 2=Ship3; 3=Ship4
                  j=Port number, i.e., 0=Zamboanga; 1=Iligan; 2=CDO; 3=Davao
            
            
    Cij = cost matrix denoting cost value for assigning ship i to port j
    
    
##### Objective:
    
    Z = total cost of all shipping
    Z = sum(Vij * Cij) for i=0...3, j=0...4
    
    
##### Constraints:
    
    sum_column(Vij) == sum_row(Vij)  # total demand == total supply
    Vij >= 0, binary (with value 1 or 0)
    
    
    
            
            

In [1]:

"""
Solve using CVXPY

"""


# import required packages
import numpy as np
import cvxpy as cvx


# Declare constant costs. 
C = cvx.Constant(np.array([[500000, 400000, 600000, 700000],
                           [600000, 600000, 700000, 500000],
                           [700000, 500000, 700000, 600000],
                           [500000, 400000, 600000, 600000]]))


# declare variable
V = cvx.Variable((4,4), boolean=True)  # the variable will have values of either 1 or 0

# Declare constraints
# Each ship shall be assigned to one specific port
row_sum = cvx.sum(V, axis=1, keepdims=True)
total_shipment = cvx.Constant(np.ones((4,1)))
constraint1 = (row_sum == total_shipment)

# Each port shall receive only one ship
col_sum = cvx.sum(V, axis=0, keepdims=True)
total_ports = cvx.Constant(np.ones((1,4)))
constraint2 = (col_sum==total_ports)

# no negative values
zeros = np.zeros((4,4),dtype=int)
constraint3 = (V>=zeros)

constraints = [constraint1,constraint2,constraint3]

# Objective function
Y = cvx.multiply(C,V)  # multiply the binary matrix with cost matrix
Z = cvx.sum(Y)  # sum all entries of the updated cost matrix

objective = cvx.Minimize(Z) # minimize the sum of all entries in the updated cost matrix
prob = cvx.Problem(objective, constraints)

# solve using different solvers
available_solvers = cvx.installed_solvers()  # fetch all available solvers 
print('Solving using:',available_solvers)
'''
Note: other solvers are available to be installed separately:

               LP  QP  SOCP  SDP   EXP  MIP
CBC            X                        X
GLPK           X
GLPK_MI        X                        X
OSQP           X   X
CPLEX          X   X  
Elemental      X   X    X
ECOS           X   X    X           X 
ECOS_BB        X   X    X           X   X
GUROBI         X   X    X               X
MOSEK          X   X    X    X
CVXOPT         X   X    X    X      X 
SCS            X   X    X    X      X 


'''


for s in available_solvers:
    print("----------------------------------------------------------")
    try:
        prob.solve(solver=s)
        print('\nOptimal value using ', s,':', np.around(prob.value, 2))
        print('Port: Zamboanga, Iligan, CDO, Davao')
        
        for i in range(0,len(V.value)):
            print("Ship ", i, ": ", np.around(V.value[i],0).astype(int))
#         print('Updated cost matrix:')
#         print(np.around(Y.value, 2).astype(int))
    except:
        print('\nCan not solve using', s)



Solving using: ['ECOS', 'ECOS_BB', 'SCS', 'OSQP']
----------------------------------------------------------

Can not solve using ECOS
----------------------------------------------------------

Optimal value using  ECOS_BB : 2100000.0
Port: Zamboanga, Iligan, CDO, Davao
Ship  0 :  [0 1 0 0]
Ship  1 :  [0 0 0 1]
Ship  2 :  [0 0 1 0]
Ship  3 :  [1 0 0 0]
----------------------------------------------------------

Can not solve using SCS
----------------------------------------------------------

Can not solve using OSQP


## Conclusion
The ECOS solver has a clearer solution which can be summarized as follows:
    
    Ship 1 should be sent to Iligan.
    Ship 2 should be sent to Davao.
    Ship 3 should be sent to Cagayan de Oro.
    Ship 4 should be sent to Zamboanga.
    
Optimum total cost of all shipments is 2,100,000.